# View Fors2 , View spectra one by one

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2022/12/23
- update : 2023/03/20
- FORS2 : https://github.com/ericnuss/fors2
- Result of StarLight : /sps/lsst/groups/photoz/fors2/FORS2/ResStarlight



          /Users/dagoret/MacOSX/GitHub/LSST/FORS2>ls -l
              total 0
              drwxrwsr-x  3 dagoret  staff   96 Aug 21  2018 ResStarlight
              drwxr-xr-x  9 dagoret  staff  288 Dec 19 18:01 fors2

## Import

In [ ]:
import os,sys,re
from astropy.io import fits
from astropy.table import Table

In [ ]:
# Import some generally useful packages

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
import pandas as pd
from itertools import cycle, islice


In [ ]:
from scipy import ndimage

#from scipy import interp as scinterp
from scipy.interpolate import interp1d

import glob
import collections
#from def_raw_seds import *

In [ ]:
# to enlarge the sizes
params = {'legend.fontsize': 'large',
          'figure.figsize': (8, 4),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

In [ ]:
import matplotlib.offsetbox
props = dict(boxstyle='round',edgecolor="w",facecolor="w", alpha=0.5)

In [ ]:
from def_raw_seds import *

In [ ]:
from raw_data_analysis import *

In [ ]:
from spectroscopy import *

In [ ]:
# Import the primary photometry classes from rubin_sim.photUtils

import rubin_sim.phot_Utils.Bandpass as Bandpass
import rubin_sim.phot_Utils.Sed as Sed
from rubin_sim.data import get_data_dir

In [ ]:
#pysynphot_root_path=os.environ['PYSYN_CDBS']
#path_sed_calspec=os.path.join(pysynphot_root_path,'calspec')
# pysynphot
#import pysynphot as S
#S.refs.setref(area=1)
#S.refs.getref()

## Configuration

### Emission lines

In [ ]:
df_lines=pd.read_excel("datatools/GalEmissionLines.xlsx")

In [ ]:
df_lines

In [ ]:
df_sdss_lines = pd.read_excel("datatools/sdss_galaxylines.xlsx")

In [ ]:
df_sdss_lines

# Lines

In [ ]:
lines_to_plot={}

In [ ]:
def plot_line(ax,line_name):
    
    #keys = lines_to_plot.keys()
    #for the_key in keys:
    #    if the_key in line_name:
    #        print("the_key=",the_key," line_name=",line_name)
    #        line_name=the_key
    #        break
            
    posx0=0.0
    posy0=0.05
    fontsize=12
    the_color = {"emission":'r',"absorption":"g","break":"purple"}
    the_line_to_plot = lines_to_plot[line_name]
    wls = the_line_to_plot["wls"]
    tag = the_line_to_plot["name"]
    typ = the_line_to_plot["type"]
    
    xlim = ax.get_xlim()
    for wl in wls:
        ax.axvline(wl, lw=2, color=the_color[typ])
        xpos = (wl - xlim[0]) / (xlim[1] - xlim[0])+posx0 
        if 0 < xpos < 1:
            ax.annotate(tag, xy=(xpos, posy0), rotation=0, ha='left', va='bottom',
                            xycoords='axes fraction', color=the_color[typ], fontsize=fontsize)
    return ax

In [ ]:
sel = df_sdss_lines["species"] == 'K'
wls = df_sdss_lines[sel]["wl"].values
Kline = Line(wls, atmospheric=False, label='$K$', label_pos=[-0.04, 0.02], use_for_calibration=False)
lines_to_plot["K"]={"wls":wls,"name":"K","type":"absorption"}

In [ ]:
sel = df_sdss_lines["species"] == 'H'
wls = df_sdss_lines[sel]["wl"].values
Hline = Line(wls, atmospheric=False, label='$H$', label_pos=[-0.04, 0.02], use_for_calibration=False)
lines_to_plot["H"]={"wls":wls,"name":"H","type":"absorption"}

In [ ]:
sel = df_sdss_lines["species"] == 'G'
wls = df_sdss_lines[sel]["wl"].values
Gline = Line(wls, atmospheric=False, label='$G$', label_pos=[-0.04, 0.02], use_for_calibration=False)
lines_to_plot["G"]={"wls":wls,"name":"G","type":"absorption"}

In [ ]:
sel = df_sdss_lines["species"] == 'Mg'
wls = df_sdss_lines[sel]["wl"].values
Gline = Line(wls, atmospheric=False, label='$Mg$', label_pos=[-0.04, 0.02], use_for_calibration=False)
lines_to_plot["Mg"]={"wls":wls,"name":"Mg","type":"absorption"}

In [ ]:
sel = df_sdss_lines["species"] == 'Na'
wls = df_sdss_lines[sel]["wl"].values
Gline = Line(wls, atmospheric=False, label='$Na$', label_pos=[-0.04, 0.02], use_for_calibration=False)
lines_to_plot["Na"]={"wls":wls,"name":"Na","type":"absorption"}

In [ ]:
sel = df_lines["ion"] == 'H8'
wls = df_lines[sel]["wl"].values
H8line = Line(wls, atmospheric=False, label='$H8$', label_pos=[-0.04, 0.02], use_for_calibration=False)
lines_to_plot["H8"]={"wls":wls,"name":"$H8$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'H9'
wls = df_lines[sel]["wl"].values
H9line = Line(wls, atmospheric=False, label='$H9$', label_pos=[-0.04, 0.02], use_for_calibration=False)
lines_to_plot["H9"]={"wls":wls,"name":"$H9$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'H10'
wls = df_lines[sel]["wl"].values
H10line = Line(wls, atmospheric=False, label='$H10$', label_pos=[-0.04, 0.02], use_for_calibration=False)
lines_to_plot["H10"]={"wls":wls,"name":"$H10$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'H11'
wls = df_lines[sel]["wl"].values
lines_to_plot["H11"]={"wls":wls,"name":"$H11$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Hα' 
wls=df_lines[sel]["wl"].values
Halphaline = Line(wls, atmospheric=False, label='$H_\\alpha$', label_pos=[-0.04, 0.02], use_for_calibration=False)
lines_to_plot["H{alpha}"]={"wls":wls,"name":"$H_\\alpha$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Hβ' 
wls=df_lines[sel]["wl"].values
Hbetaline = Line(wls, atmospheric=False, label='$H_\\beta$', label_pos=[-0.04, 0.02], use_for_calibration=False)
lines_to_plot["H{beta}"]={"wls":wls,"name":"$H_\\beta$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Hγ' 
wls=df_lines[sel]["wl"].values
lines_to_plot["H{gamma}"]={"wls":wls,"name":"$H_\\gamma$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Hδ' 
wls=df_lines[sel]["wl"].values
lines_to_plot["H{delta}"]={"wls":wls,"name":"$H_\\delta$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Hε' 
wls=df_lines[sel]["wl"].values
lines_to_plot["H{epsilon}"]={"wls":wls,"name":"$H_\\epsilon$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == '[O II]'
wls=df_lines[sel]["wl"].values
lines_to_plot["[OII]"]={"wls":wls,"name":"$[OII]$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == '[O III]'
wls=df_lines[sel]["wl"].values
lines_to_plot["[OIII]"]={"wls":wls,"name":"$[OIII]$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'O IV]'
wls=df_lines[sel]["wl"].values
lines_to_plot["[OIV]"]={"wls":wls,"name":"$[OIV]$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'O VI'
wls=df_lines[sel]["wl"].values
lines_to_plot["[OVI]"]={"wls":wls,"name":"$[OVI]$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Mgb'
wls=df_lines[sel]["wl"].values
lines_to_plot["Mgb"]={"wls":wls,"name":"$Mgb$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Mg II]'
wls=df_lines[sel]["wl"].values
lines_to_plot["MgII"]={"wls":wls,"name":"$MgII$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Fe43'
wls=df_lines[sel]["wl"].values
lines_to_plot["Fe43"]={"wls":wls,"name":"$Fe43$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Fe45'
wls=df_lines[sel]["wl"].values
lines_to_plot["Fe45"]={"wls":wls,"name":"$Fe45$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Ca44'
wls=df_lines[sel]["wl"].values
lines_to_plot["Ca44"]={"wls":wls,"name":"$Ca44$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'E'
wls=df_lines[sel]["wl"].values
lines_to_plot["E"]={"wls":wls,"name":"$E$","type":"emission"}

In [ ]:
sel = df_lines["ion"] =='Fe II'
wls=df_lines[sel]["wl"].values
lines_to_plot["FeII24"]={"wls":wls,"name":"$FeII24$","type":"emission"}
lines_to_plot['FeII26']={"wls":wls,"name":"$FeII26$","type":"emission"}

In [ ]:
lines_to_plot['weak']={"wls":[],"name":"$weak$","type":"break"}
lines_to_plot['?']={"wls":[],"name":"$?$","type":"break"}

In [ ]:
lines_to_plot['4000{AA}-break']={"wls":[4000.],"name":"$Bal$","type":"break"}

In [ ]:
sel = df_lines["ion"] == 'Lyα'
wls=df_lines[sel]["wl"].values
lines_to_plot["Ly{alpha}"]={"wls":wls,"name":"$Ly_\\alpha$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Lyβ'
wls=df_lines[sel]["wl"].values
lines_to_plot["Ly{beta}"]={"wls":wls,"name":"$Ly_\\beta$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Lyδ'
wls=df_lines[sel]["wl"].values
lines_to_plot["Ly{delta}"]={"wls":wls,"name":"$Ly_\\delta$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Lyε'
wls=df_lines[sel]["wl"].values
lines_to_plot["Ly{epsilon}"]={"wls":wls,"name":"$Ly_\\epsilon$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'C IV'
wls=df_lines[sel]["wl"].values
lines_to_plot["CIV"]={"wls":wls,"name":"$CIV$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Al III'
wls=df_lines[sel]["wl"].values
lines_to_plot["AlIII"]={"wls":wls,"name":"$AlIII$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == '[Ne III]'
wls=df_lines[sel]["wl"].values
lines_to_plot['NeIII']={"wls":wls,"name":"$NeIII$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'He I'
wls=df_lines[sel]["wl"].values
lines_to_plot['HeI']={"wls":wls,"name":"$HeI$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'N III'
wls=df_lines[sel]["wl"].values
lines_to_plot['NIII']={"wls":wls,"name":"$NIII$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Al II'
wls=df_lines[sel]["wl"].values
lines_to_plot['AlII']={"wls":wls,"name":"$AlII$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'Al III'
wls=df_lines[sel]["wl"].values
lines_to_plot['AlIII']={"wls":wls,"name":"$AlIII$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == '[N II]'
wls=df_lines[sel]["wl"].values
lines_to_plot['NII']={"wls":wls,"name":"$NII$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'C III'
wls=df_lines[sel]["wl"].values
lines_to_plot['CIII']={"wls":wls,"name":"$CIII$","type":"emission"}

In [ ]:
sel = df_lines["ion"] == 'C IV'
wls=df_lines[sel]["wl"].values
lines_to_plot['CIV']={"wls":wls,"name":"$CIV$","type":"emission"}

In [ ]:
sel = df_sdss_lines["species"] == 'Si IV + O IV'
wls=df_sdss_lines[sel]["wl"].values
lines_to_plot['SiIV/OIV']={"wls":wls,"name":"$SiIV/OIV$","type":"emission"}

In [ ]:
lines_to_plot["(QSO)"] = {"wls":[],"name":"$QSO$","type":"emission"}
lines_to_plot["QSO"] = {"wls":[],"name":"$QSO$","type":"emission"}

In [ ]:
lines_to_plot['NaD'] = {"wls":[],"name":"$NaD$","type":"emission"}

In [ ]:
lines_to_plot['broad'] = {"wls":[],"name":"$broad$","type":"emission"}

In [ ]:
lines_to_plot

### Fors2

## Start

### Catalog

In [ ]:
t = Table.read(filename_fits_catalog)

In [ ]:
t

### List

In [ ]:
lst=fors2_jct_SL() #remet les spectres dans le rest frame (ie a z=0)

In [ ]:
lst = sorted(lst)
print(lst)

In [ ]:
all_specfilenames = os.listdir(path_out_jct)

In [ ]:
all_specfilenames = sorted(all_specfilenames)

In [ ]:
all_numbers = [ int(re.findall('^SPEC(.*)[.]txt$',fn)[0]) for fn in all_specfilenames ]

In [ ]:
all_numbers = sorted(all_numbers)

In [ ]:
print(all_numbers)

In [ ]:
NSED = len(all_numbers)

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NSED)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
cut = t["ID"] == 3
t[cut]

In [ ]:
t[cut]["z"].value[0]

In [ ]:
all_spectra = []
all_wl = []
all_tag = []
all_lines = []
#all_sed = []

for the_num in all_numbers:
    tcut = t["ID"] == the_num
    redshift = t[tcut]["z"].value[0]
    lines = t[tcut]["Lines"].value[0]
    filename = f"SPEC{the_num}.txt"
    tag = f"SPEC{the_num} , z={redshift:.2f}, lines={lines}"
    fullfilename = os.path.join(path_out_jct,filename)
#    the_sed=SED_jct(the_num)
#    all_sed.append(the_sed)
    arr = np.loadtxt(fullfilename)
    all_wl.append(arr[:,0])
    all_spectra.append(arr[:,1])
    all_tag.append(tag)
    all_lines.append(lines)
 

In [ ]:
if 0:
    for idx in range(NSED):
        plt.figure(figsize=(16,2))
        plt.plot(all_wl[idx],all_spectra[idx], color=all_colors[idx],label=all_tag[idx])
        plt.legend(loc="upper right")
        plt.grid()
        plt.axvline(4150,color="k")
        plt.axvline(4250,color="k")
#    plt.ylim(-0.02,0.4)
        plt.xlim(2000.,8000.)
        plt.show()


In [ ]:
idx=82
the_lines = all_lines[idx]
print("the_lines = ",the_lines)
all_elements  = the_lines.decode().split(",")
#fig, ax = plt.subplot(1,1,1,figsize=(16,8))
fig, axs = plt.subplots(1, 1, figsize=(16, 8),dpi=50)
ax=axs
ax.plot(all_wl[idx],all_spectra[idx],'b-' ,color=all_colors[idx],label=all_tag[idx],lw=3)
ax.legend(loc="upper right")
ax.grid()
ax.axvline(4150,color="grey")
ax.axvline(4250,color="grey")
#    plt.ylim(-0.02,0.4)
ax.set_xlim(2000.,8000.)
for el in all_elements:
    el=el.split(" ")[0]  
    print(el)
    if len(el)>0:
        plot_line(ax,el)


In [ ]:
for idx in range(515,NSED):
    the_tag=all_tag[idx] 
    print(f"===================== SED {idx} : {the_tag}=======================")
    
    the_lines = all_lines[idx]
    print("the_lines = ",the_lines)
    all_elements  = the_lines.decode().split(",")
    #fig, ax = plt.subplot(1,1,1,figsize=(16,8))
    fig, axs = plt.subplots(1, 1, figsize=(16, 8),dpi=50)
    ax=axs
    ax.plot(all_wl[idx],all_spectra[idx],'b-' ,color=all_colors[idx],label=all_tag[idx],lw=3)
    ax.legend(loc="upper right")
    ax.grid()
    ax.axvline(4150,color="grey")
    ax.axvline(4250,color="grey")
#    plt.ylim(-0.02,0.4)
    ax.set_xlim(2000.,8000.)
    for el in all_elements:
        el=el.split(" ")[0]  
        el=el.split("\r")[0]  
        print(el)
        if len(el)>0:
            plot_line(ax,el)
    plt.show()
    
    

In [ ]:
NSED